<a href="https://colab.research.google.com/github/cku7808/kdata-MLDL-mini-project/blob/main/LSTM_%EC%B5%9C%EA%B3%A0%EC%9A%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
!pip3 install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

os.chdir('/content/drive/Shareddrives/B반 3조')
%pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/Shareddrives/B반 3조'

In [ ]:
train_data = pd.read_csv('7 22 머신러닝 딥러닝 프로젝트/data/naver_data.csv', header = 0, delimiter=',')
test_data = pd.read_csv('7 22 머신러닝 딥러닝 프로젝트/data/test.csv', header = 0, delimiter=',')

train_data

,ID,날짜,제목,조회,공감,비공감,label
0,0,2022-07-20 15:31,코인시장 물반 고기반,45,0,0,-1
1,1,2022-07-20 15:31,조선장 지긋지긋하다 ㅋㅋ,27,0,0,0
2,2,2022-07-20 14:51,네이버 나스닥 이전 상장 하면,231,0,0,-1
3,3,2022-07-20 14:51,시간이 다 됐다..빨리 빨리 올라가자,60,0,0,1
4,4,2022-07-20 14:47,코피터진 jp,79,0,0,0
...,...,...,...,...,...,...,...
5975,5975,2022-05-20 9:20,ㅅ블년,158,14,0,0
5976,5976,2022-05-20 9:19,41층입니다.,373,2,0,-1
5977,5977,2022-05-20 9:18,니 굴파고 들어갈라꼬 폼잡나,101,2,0,-1
5978,5978,2022-05-20 9:18,혼자 파랭이 실화냐?,98,4,0,0


In [ ]:
n=train_data['label'].isin([' ','-1']) #공백과 -1이 포함된 요소 찾기
train_data = train_data[~n]

In [ ]:
train_data[['label']] = train_data[['label']].astype(int)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
train_data

,ID,날짜,제목,조회,공감,비공감,label
1,1,2022-07-20 15:31,조선장 지긋지긋하다 ㅋㅋ,27,0,0,0
3,3,2022-07-20 14:51,시간이 다 됐다..빨리 빨리 올라가자,60,0,0,1
4,4,2022-07-20 14:47,코피터진 jp,79,0,0,0
7,7,2022-07-20 14:31,우리나라 왠만한 기업들 국장 버리고,80,1,1,0
11,11,2022-07-20 14:26,바지사장 대표랑 뒤에 숨은 이해찬 독하다. 이것들 진짜 아무 것도 안하네.,51,0,0,0
...,...,...,...,...,...,...,...
5973,5973,2022-05-20 9:22,네이버에 물릴날이 오다니 ㅠㅠ,128,1,0,0
5974,5974,2022-05-20 9:21,건물이랑 직원들 급여 올리면서,133,5,0,0
5975,5975,2022-05-20 9:20,ㅅ블년,158,14,0,0
5978,5978,2022-05-20 9:18,혼자 파랭이 실화냐?,98,4,0,0


In [ ]:
train_data.shape

(2843, 7)

# 1.1훈련데이터 전처리 

# 특수문자 지우기

문제) 숫자 표현을 살릴까 말까


In [ ]:
#Null값 탐색
print(train_data.isnull().values.any())
print(train_data.isnull().sum())

False
ID       0
날짜       0
제목       0
조회       0
공감       0
비공감      0
label    0
dtype: int64


In [ ]:
#한글과 공백만 선택
train_data['제목'] = train_data['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #ㄱ-ㅎㅏ-ㅣ가-힣 가 아닌 문자는 공백처리
train_data["제목"] = train_data["제목"].replace("\n","").replace('\t','').replace('\r','')

train_data[:10]

#숫자 표현도 고려할 시 아래 정규식 사용
'''
train_data['제목'] = train_data['제목'].str.replace("[-=■●+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]","")
train_data['제목'] = train_data['제목'].str.replace("[a-zA-Z]","")
'''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


'\ntrain_data[\'제목\'] = train_data[\'제목\'].str.replace("[-=■●+,#/\\?:^.@*"※~ㆍ!』‘|\\(\\)\\[\\]`\'…》\\”\\“\\’·]","")\ntrain_data[\'제목\'] = train_data[\'제목\'].str.replace("[a-zA-Z]","")\n'

In [ ]:
#Null값 탐색
print(train_data.isnull().values.any())
print(train_data.isnull().sum())

cnt=0
n = train_data.copy()
for i in range(len(n)):
  if n.iloc[i]['제목'] == "" :
    cnt += 1
    train_data = train_data.drop(train_data.index[i])

print(len(n))
print(cnt)
print(len(train_data))

train_data.shape

False
ID       0
날짜       0
제목       0
조회       0
공감       0
비공감      0
label    0
dtype: int64
2843
2
2841


(2841, 7)

In [ ]:
print('남은 Row 수 :',len(train_data)) # 2840 개 나오면 됨!

남은 Row 수 : 2841


# 1.2 테스트데이터 전처리 

In [ ]:
#Null값 탐색
print(test_data.isnull().values.any())
print(test_data.isnull().sum())

cnt = 0
m = test_data.copy()
for i in range(len(m)):
  if m.iloc[i]['제목'] == "" :
    cnt += 1
    test_data = test_data.drop(test_data.index[i])

print(len(m))
print(cnt)
print(len(test_data))

False
Unnamed: 0    0
날짜            0
제목            0
조회            0
공감            0
비공감           0
dtype: int64
840
0
840


In [ ]:
test_data.shape

(840, 6)

In [ ]:
#한글과 공백만 선택
test_data['제목'] = test_data['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #ㄱ-ㅎㅏ-ㅣ가-힣 에 포함되지 않는 문자는 ''로 대체
test_data["제목"] = test_data["제목"].replace("\n","").replace('\t','').replace('\r','')

test_data[:10]

#숫자 표현도 고려할 시 아래 정규식 사용
'''
train_data['제목'] = train_data['제목'].str.replace("[-=■●+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]","")
train_data['제목'] = train_data['제목'].str.replace("[a-zA-Z]","")'''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


'\ntrain_data[\'제목\'] = train_data[\'제목\'].str.replace("[-=■●+,#/\\?:^.@*"※~ㆍ!』‘|\\(\\)\\[\\]`\'…》\\”\\“\\’·]","")\ntrain_data[\'제목\'] = train_data[\'제목\'].str.replace("[a-zA-Z]","")'

In [ ]:

print('남은 Row 수 :',len(test_data)) # 840 개 나오면 됨!


남은 Row 수 : 840


# 형태소 분석(토큰화)
-> 불용어 선택

-> 형태소 분석기 선택(okt, kkoma, hannanum)

-> 정확도는 kkoma가 가장 좋은 듯...?



In [ ]:
# 학습 데이터가 비교적 작고 도메인도 좁기 때문에 최대한 조사와 같은 기능어 중심으로 편성함
# stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
clean_train_subject = []

In [ ]:
# 형태소 분석기: "okt"
def preprocessing_okt(subject, okt, remove_stopwords = False, stop_words =[]):#
  # 형태소 분석
  # okt는 별도로 스테머를 활용 가능
  subject_word = okt.morphs(subject,stem=True)
  if remove_stopwords:
    # 불용어 제거
    subject_word = [token for token in subject_word if not token in stop_words]
  return subject_word

# 형태소 분석기: "kkma" 
# 가장 오래 걸림: 1분 47초 소요
def preprocessing_kkma(subject, kkma, remove_stopwords = False, stop_words =[]):#
  # 형태소 분석
  subject_word = kkma.morphs(subject)
  if remove_stopwords:
    # 불용어 제거
    subject_word = [token for token in subject_word if not token in stop_words]
  return subject_word

# 형태소 분석기: "hannanum"
def preprocessing_hannanum(subject, hannanum, remove_stopwords = False, stop_words =[]):#
  # 형태소 분석
  subject_word = hannanum.morphs(subject)
  if remove_stopwords:
    # 불용어 제거
    subject_word = [token for token in subject_word if not token in stop_words]
  return subject_word

# 형태소 분석기: "komoran"
def preprocessing_komoran(subject, komoran, remove_stopwords = False, stop_words =[]):#
  # 형태소 분석
  subject_word = komoran.morphs(subject)
  if remove_stopwords:
    # 불용어 제거
    subject_word = [token for token in subject_word if not token in stop_words]
  return subject_word

In [ ]:
# 형태소 분석기 선택하기
order = input('사용할 형태소 분석기를 선택(1: okt, 2: kkma, 3: hannanum, 4: komoran)\n>>')
clean_train_subject = []
if order == '1':
  print('okt 실행 중')
  # 불용어 리스트
  stop_words = '다 은 는 이 가 하 아 것 들 의 있 자 되 수 보 주 등 고 한'.split()
  okt = Okt()
  for subject in train_data['제목']:
    clean_train_subject.append(preprocessing_okt(subject,okt,remove_stopwords=True,stop_words= stop_words))
  print(clean_train_subject[:10])

elif order == '2':
  print('kkma 실행 중')
  # 불용어 리스트
  stop_words = '이 있 하 다 것 ㄴ 들 ㄹ라고 ㄹ ㄹ지 게 그 되 수 ㅂ니다 ㄴ다 여 ㅁ 나 아니 는 았 등 같 때 러 가 한 지 오 에 말 일'.split()
  kkma = Kkma()
  try:
    for subject in train_data['제목']:
      clean_train_subject.append(preprocessing_kkma(subject,kkma,remove_stopwords=True,stop_words= stop_words))
    print(clean_train_subject[:100])
  except:
    pass

elif order == '3':
  print('hannanum 실행 중')
  # 불용어 리스트
  stop_words = '자 ㅁ 가 은 고 는 이 하 ㅂ니다 아 것 들 ㄴ 다 의 었 있 되 수 보 주 등 한'.split()
  hannanum = Hannanum()
  for subject in train_data['제목']:
    clean_train_subject.append(preprocessing_hannanum(subject,hannanum,remove_stopwords=True,stop_words= stop_words))
  print(clean_train_subject[:100])

elif order == '4':
  print('komoran 실행 중')
  # 불용어 리스트
  stop_words = '자 은 고 는 니 이 었 ㄴ다 가 하 니까 ㅂ니다 아 것 ㅁ 들 ㄴ 다 의 있 되 수 보 ㄴ다고 주 등 면 ㄴ가 한'.split()
  komoran = Komoran()
  for subject in train_data['제목']:
    clean_train_subject.append(preprocessing_komoran(subject,komoran,remove_stopwords=True,stop_words= stop_words))
  print(clean_train_subject[:100])



사용할 형태소 분석기를 선택(1: okt, 2: kkma, 3: hannanum, 4: komoran)
>>2
kkma 실행 중


# 테스트 데이터 토큰화

In [ ]:
# 형태소 분석기 선택하기
# order = input('사용할 형태소 분석기를 선택(1: okt, 2: kkma, 3: hannanum, 4: komoran)\n>>')
X_test = []
if order == '1':
  print('okt 실행 중')
  # 불용어 리스트
  stop_words = '다 은 는 이 가 하 아 것 들 의 있 자 되 수 보 주 등 고 한'.split()
  okt = Okt()
  for subject in train_data['제목']:
    X_test.append(preprocessing_okt(subject,okt,remove_stopwords=True,stop_words= stop_words))
  print(X_test[:10])

elif order == '2':
  print('kkma 실행 중')
  # 불용어 리스트
  stop_words = '이 있 하 다 것 ㄴ 들 ㄹ라고 ㄹ ㄹ지 게 그 되 수 ㅂ니다 ㄴ다 여 ㅁ 나 아니 는 았 등 같 때 러 가 한 지 오 에 말 일'.split()
  kkma = Kkma()
  try:
    for subject in train_data['제목']:
      X_test.append(preprocessing_kkma(subject,kkma,remove_stopwords=True,stop_words= stop_words))
    print(X_test[:100])
  except:
    pass

elif order == '3':
  print('hannanum 실행 중')
  # 불용어 리스트
  stop_words = '자 ㅁ 가 은 고 는 이 하 ㅂ니다 아 것 들 ㄴ 다 의 었 있 되 수 보 주 등 한'.split()
  hannanum = Hannanum()
  for subject in train_data['제목']:
    X_test.append(preprocessing_hannanum(subject,hannanum,remove_stopwords=True,stop_words= stop_words))
  print(X_test[:100])

elif order == '4':
  print('komoran 실행 중')
  # 불용어 리스트
  stop_words = '자 은 고 는 니 이 었 ㄴ다 가 하 니까 ㅂ니다 아 것 ㅁ 들 ㄴ 다 의 있 되 수 보 ㄴ다고 주 등 면 ㄴ가 한'.split()
  komoran = Komoran()
  for subject in train_data['제목']:
    X_test.append(preprocessing_komoran(subject,komoran,remove_stopwords=True,stop_words= stop_words))
  print(X_test[:100])


kkma 실행 중


# 인덱스 벡터로 변환


In [ ]:
#단어 등장 빈도에 따른 정수값 부여
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_subject) #fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다

In [ ]:
#사전을 보여줌(빈도순임)
print(tokenizer.word_index)

In [ ]:
threshold = 3
word_vocab = tokenizer.word_index # 단어 사전 
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

https://wikidocs.net/44249 에 따르면...

<okt기준>
훈련 데이터에서 등장 빈도가 2회 이하인 단어는 단어 집합에서 절반 이상을 차지 중. 하지만, 실제로 훈련 데이터에서 등장 빈도로 차지하는 비중은 상대적으로 비교적 적은 수치인 8.44. 따라서 2회 이하인 단어들은 자연어 처리에서 별로 중요하지 않다고 볼 수도 있음. 그래서 이 단어들은 정수 인코딩 과정에서 배제하는 것도 충분히 가능함.

다만, 8.75는 참고 자료에서 제시한 비율보다 더 높은 수준이라 일단 배제하지는 않겠음.

In [ ]:
vocab_size = total_cnt
print('단어 집합의 크기 :',vocab_size)

### 텍스트 시퀀스를 정수 시퀀스로 변환

In [ ]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(clean_train_subject) # 앞에서 토큰화가 완료된 단어집
X_train = tokenizer.texts_to_sequences(clean_train_subject) #문자열을 인덱스의 리스트로 변환
X_test = tokenizer.texts_to_sequences(X_test) #테스트 데이터 생성시 활용

In [ ]:
print(X_train[:5])

# 단어 패딩

-> 학습 모델이 처리할 수 있도록 모든 문장들을 동일한 길이의 차원의 벡터로 담아야 함. 가장 길이가 긴 문장을 담으면 좋겠지만, 그럴 경우 불필요하게 과다한 차원이 생겨 작업 효율을 저하시킬 수 있기 때문에 전체적인 데이터의 크기를 볼 필요가 있음

-> 데이터 시각화로 전체적인 문자열의 길이를 확인, 적당한 크기를 차원수로 확정

# 주의!
### 형태소 분석기(불용어 필터)에 따라 샘플 크기가 달라지기 때문에 이 부분에서는 유동적으로 max_len의 크기를 조절하는 게 좋음

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print('제목의 최대 길이 :',max(len(review) for review in X_train))
print('제목의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
#전체 문장들에서 특정 길이 이하인 문장들의 비율을 제시해주는 함수
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 13
below_threshold_len(max_len, X_train)

In [ ]:
# n차원의 벡터로 패딩하기
train_inputs = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
#학습 데이터 라벨 벡터화
train_labels = np.array(train_data['label'])

In [ ]:
#drop_train에는 X_train으로부터 얻은 빈 샘플들의 인덱스가 저장
drop = [index for index, sentence in enumerate(train_inputs) if len(sentence) < 1]

# 빈 샘플들을 제거
train_inputs = np.delete(train_inputs, drop, axis=0)
train_labels = np.delete(train_labels, drop, axis=0)
print(len(train_inputs)) # 샘플의 수
print(len(train_labels)) # 샘플의 수


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_inputs, train_labels, test_size=0.2, random_state=42)

## 데이터 저장

In [ ]:
DEFAULT_PATH  = '/content/drive/Shareddrives/B반 3조/7 22 머신러닝 딥러닝 프로젝트/data/' # 경로지정
DATA_PATH = 'CLEAN_DATA/' #.npy파일 저장 경로지정
TRAIN_INPUT_DATA = 'community_train_input.npy'
TRAIN_LABEL_DATA = 'community_train_label.npy'
TEST_INPUT_DATA = 'community_test_input.npy'
TEST_LABEL_DATA = 'community_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs={}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

#전처리한 데이터들 파일로저장
import os

if not os.path.exists(DEFAULT_PATH + DATA_PATH):
  os.makedirs(DEFAULT_PATH+DATA_PATH)

#전처리 학습데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)
#전처리 테스트데이터 넘파이로 저장


#데이터 사전 json으로 저장
json.dump(data_configs,open(DEFAULT_PATH + DATA_PATH + DATA_CONFIGS,'w'),ensure_ascii=False)

## 모델 구현

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

## 모델학습

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_val, y_val)[1]))

#형태소 분석기 별 정확도
# 1. okt : 정확도 86%
# 2. kkma : 정확도 88%
# 3. hannanum : 정확도 84%
# 4. komoran : 정확도 87%

In [ ]:
def sentiment_predict(new_sentence):
  original = new_sentence
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True)
  new_sentence = [word for word in new_sentence if not word in stop_words]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)
  score = float(loaded_model.predict(pad_new))

  #print(original)
  if(score > 0.5):
    #print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    return True
  else:
    #print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1-score) * 100))
    return False


test_data.head()

In [ ]:
# j = 0
# for title in test_data["제목"]:
#   if(j > 10):
#     break
#   sentiment_predict(title)
#   j+=1

In [ ]:
r_positive = 0
r_negative = 0

for title in test_data["제목"]:
  if sentiment_predict(title):
    r_positive += 1
  else:
    r_negative += 1

if r_positive > r_negative:
  print("{:.2f}%의 확률로 상승".format(r_positive/len(test_data["제목"]) * 100))
else:
  print("{:.2f}%의 확률로 하락".format(r_negative/len(test_data["제목"]) * 100))